$n$ is the number of clientes

$N$ is set of clients, with $N=\{1,2,...,n\}$

$V$ is set of vetices (or nodes), with $V=\{0\}\cup N$

$A$ is set of arcs, with $A=\{(i,j)\in V^2 : i\neq j\}$

$c_{ij}$ is cost of travel over arc $(i,j)\in A$

$Q$ is the vehicle capacity

$q_i$ is the amount that has to be delivered to customer $i\in N$

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from docplex.mp.model import Model
from matplotlib.pyplot import figure
%matplotlib inline
import folium
import gmaps
import gmplot

In [2]:
features = pd.read_excel("C:/Users/Dell/Desktop/datasender.xls")
features.head(5)

,Sender Name,Start lat,Start lan,Vehicle Capacity,Recipient Name,Recipient lat,Recipient lan,Amount delivery
0,Sen1,33.590401,-7.620526,20.0,Rec1,33.587907,-7.617337,6
1,NaN,NaN,NaN,NaN,Rec2,33.566510,-7.578457,3
2,NaN,NaN,NaN,NaN,Rec3,33.552353,-7.553179,4
3,NaN,NaN,NaN,NaN,Rec4,33.537164,-7.576377,4
4,NaN,NaN,NaN,NaN,Rec5,33.578449,-7.649248,8


In [3]:
features.count()

Sender Name          1
Start lat            1
Start lan            1
Vehicle Capacity     1
Recipient Name      10
Recipient lat       10
Recipient lan       10
Amount delivery     10
dtype: int64

In [4]:
n = features['Amount delivery'].count()
Q = features['Vehicle Capacity'].sum()
N1=[]
for i in range (1,n+1):
    N1.append(features['Recipient Name'][i-1])
V1 = [features['Sender Name'][0]]+N1


loc_x = []
loc_y =[]
loc_x.append(round(features['Start lat'][0],7))
loc_y.append(round(features['Start lan'][0],7))

q1={}
for m in range(0, n):
    q1[features['Recipient Name'][m]]=features['Amount delivery'][m]
    loc_x.append(round(features['Recipient lat'][m],7))
    loc_y.append(round(features['Recipient lan'][m],7))
    
print('n = ',n)
print('****')
print('Q = ',Q)
print('****')
print('N1 = \n',N1)
print('****')
print('V1 = \n',V1)
print('****')
print('q1 = \n',q1)
print('****')
print('loc_x = \n',loc_x)
print('****')
print('loc_y = \n',loc_y)


n =  10
****
Q =  20.0
****
N1 = 
 ['Rec1', 'Rec2', 'Rec3', 'Rec4', 'Rec5', 'Rec6', 'Rec7', 'Rec8', 'Rec9', 'Rec10']
****
V1 = 
 ['Sen1', 'Rec1', 'Rec2', 'Rec3', 'Rec4', 'Rec5', 'Rec6', 'Rec7', 'Rec8', 'Rec9', 'Rec10']
****
q1 = 
 {'Rec1': 6, 'Rec2': 3, 'Rec3': 4, 'Rec4': 4, 'Rec5': 8, 'Rec6': 8, 'Rec7': 10, 'Rec8': 5, 'Rec9': 15, 'Rec10': 2}
****
loc_x = 
 [33.5904014, 33.587907, 33.5665096, 33.5523534, 33.5371637, 33.5784488, 33.564814, 33.5612433, 33.5451624, 33.5623764, 33.5835463]
****
loc_y = 
 [-7.6205263, -7.6173367, -7.5784567, -7.5531788, -7.5763768, -7.6492485, -7.6560206, -7.6679681, -7.6693127, -7.6731874, -7.6072074]


In [5]:
N = []
q = {}
for i in range(1,len(N1)+1):
    N.append(i)
    q[i]=features['Amount delivery'][i-1]
    
Q=int(Q)    
V=[0]+N

In [6]:
A = [(i, j) for i in V for j in V if i != j]
c = {(i, j): np.hypot(loc_x[i]-loc_x[j], loc_y[i]-loc_y[j]) for i, j in A}
mdl = Model('CVRP')
x = mdl.binary_var_dict(A, name='x')
u = mdl.continuous_var_dict(N, ub=Q, name='u')
mdl.minimize(mdl.sum(c[i, j]*x[i, j] for i, j in A))
mdl.add_constraints(mdl.sum(x[i, j] for j in V if j != i) == 1 for i in N)
mdl.add_constraints(mdl.sum(x[i, j] for i in V if i != j) == 1 for j in N)
mdl.add_indicator_constraints(mdl.indicator_constraint(x[i, j], u[i]+q[j] == u[j]) for i, j in A if i != 0 and j != 0)
mdl.add_constraints(u[i] >= q[i] for i in N)
mdl.parameters.timelimit = 15
solution = mdl.solve(log_output=True)
print('****')
print(solution)

Version identifier: 12.10.0.0 | 2019-11-27 | 843d4de2ae
CPXPARAM_Read_DataCheck                          1
CPXPARAM_RandomSeed                              201903125
CPXPARAM_TimeLimit                               15
Found incumbent of value 0.941325 after 0.00 sec. (0.01 ticks)
Tried aggregator 2 times.
MIP Presolve eliminated 10 rows and 8 columns.
MIP Presolve modified 41 coefficients.
Aggregator did 41 substitutions.
Reduced MIP has 63 rows, 153 columns, and 311 nonzeros.
Reduced MIP has 102 binaries, 0 generals, 0 SOSs, and 82 indicators.
Presolve time = 0.03 sec. (0.33 ticks)
Probing time = 0.00 sec. (0.29 ticks)
Tried aggregator 1 time.
Detecting symmetries...
Reduced MIP has 63 rows, 153 columns, and 311 nonzeros.
Reduced MIP has 102 binaries, 0 generals, 0 SOSs, and 82 indicators.
Presolve time = 0.01 sec. (0.25 ticks)
Probing time = 0.02 sec. (0.29 ticks)
Clique table members: 89.
MIP emphasis: balance optimality and feasibility.
MIP search method: dynamic search.
Parallel m

In [7]:
solution.solve_status
active_arcs = [a for a in A if x[a].solution_value > 0.9]
active_arcs=np.asarray(active_arcs)
active_arcs=active_arcs.tolist()
print('Active_arcs are :\n',active_arcs)

Active_arcs are :
 [[0, 4], [0, 5], [0, 7], [0, 9], [1, 0], [2, 10], [3, 2], [4, 3], [5, 0], [6, 0], [7, 6], [8, 0], [9, 8], [10, 1]]


In [8]:
import gmplot
import numpy as np

gmap = gmplot.GoogleMapPlotter(33.590401, -7.620526, 12)

for i in range (0,len(active_arcs)):
    k=active_arcs[i]
    s1=k[0]
    s2=k[1]
    path = zip(*[
    (loc_x[s1], loc_y[s1]),(loc_x[s2], loc_y[s2])
          ])
    gmap.plot(*path, edge_width=7, color='blue')
    
gmap.heatmap(loc_x, loc_y)
gmap.scatter(loc_x, loc_y, c='r', marker=True)
gmap.enable_marker_dropping('yellow', draggable=True)

gmap.apikey = "goole_maps_api_key"
gmap.draw("map.html")
#print(gmap.get())

In [9]:
M=[]
for i in active_arcs:
    T1 = i
    for j in active_arcs:
        T2 = j
        if T1[len(T1)-1]==T2[0] and T2[0]!=0 :
            T1.append(T2[1])
            active_arcs.extend([T1])
            if T1 not in M and T1[0]==0:
                M.append(T1)
print('Shortest paths are :\n', M)   

Shortest paths are :
 [[0, 4, 3, 2, 10, 1, 0], [0, 5, 0], [0, 7, 6, 0], [0, 9, 8, 0]]


# Now you can  open the file 'map.html' to see your paths